# This notebook works on extracting unique named entities and organizations from KDD papers and passing them lists.  

In [55]:
import pandas as pd
import numpy as np
import nltk
import os
import subprocess
import unicodedata
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from nltk import Tree
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
import re
from operator import itemgetter
import polyglot
import string
from emailextractor import file_to_str, get_emails # paste code to .py file from following link and save within your environment path to call it: https://gist.github.com/dideler/5219706

'''
Severl issues extended the time to complete this work
that were not related to the coding itself.  I had two
Anaconda distros installed on my computer.  This led
to problems with importing modules because the paths
for installation and retrieval were mixed.  I had to
uninstall both anacondas, reinstall, and then recreate
my virtual environment.  Imports and installs worked cleanly

To create ipython shells in a virtual environment, use:
http://stackoverflow.com/questions/30492623/using-both-python-2-x-and-python-3-x-in-ipython-notebook

''';

In [7]:
path        = os.path.abspath(os.getcwd())
TESTDIR     = os.path.normpath(os.path.join(os.path.expanduser("~"),"projects","LC3-Creations", "examples","KDDsample"))


In [8]:
'''
I experienced unicode problems early on.  Everytime I had an error, I scoured the internet for solutions. Here's the credit.



- For Typeerror codes using subprocess to convert pdf2txt output to straight unicode --> http://stackoverflow.com/questions/33283603/python-popen-communicate-str-encodeencoding-utf-8-errors-ignore-cr
- For problems with ASCII characters --> http://stackoverflow.com/questions/175240/how-do-i-convert-a-files-format-from-unicode-to-ascii-using-python
- For unicode characters left in unicode converted to a string  --> http://stackoverflow.com/questions/8689795/how-can-i-remove-non-ascii-characters-but-leave-periods-and-spaces-using-python

'''


a = unicode(subprocess.check_output(['pdf2txt.py',str(os.path.normpath(os.path.join(TESTDIR,"p1005.pdf")))]),errors='ignore')
document = filter(lambda x: x in string.printable,unicodedata.normalize('NFKD', a).encode('ascii','ignore').decode('unicode_escape').encode('ascii','ignore'))

In [9]:
document[:2000]

'Mining Frequent Itemsets through Progressive Sampling\n\nwith Rademacher Averages\n\nMatteo Riondato\n\nDept. of Computer Science\n\nBrown University\n\nProvidence, RI 02912\n\nmatteo@cs.brown.edu\n\nEli Upfal\n\nDept. of Computer Science\n\nBrown University\n\nProvidence, RI 02912\neli@cs.brown.edu\n\nABSTRACT\nWe present an algorithm to extract an high-quality approx-\nimation of the (top-k) Frequent itemsets (FIs) from ran-\ndom samples of a transactional dataset. With high prob-\nability the approximation is a superset of the FIs, and no\nitemset with frequency much lower than the threshold is in-\ncluded in it. The algorithm employs progressive sampling,\nwith a stopping condition based on bounds to the empirical\nRademacher average, a key concept from statistical learning\ntheory. The computation of the bounds uses characteris-\ntic quantities that can be obtained eciently with a sin-\ngle scan of the sample. Therefore, evaluating the stopping\ncondition is fast, and does not re

In [10]:
"returns named entity chunks in a given text"
tagged = nltk.pos_tag(nltk.word_tokenize(re.sub('[\s]'," ", document)))
entities = nltk.chunk.ne_chunk(tagged)
# Another entity extractor
st = StanfordNERTagger('/Users/linwood/stanford-corenlp-full-2015-04-20/classifiers/english.muc.7class.distsim.crf.ser.gz',
       '/Users/linwood/stanford-corenlp-full-2015-04-20/stanford-corenlp-3.5.2.jar',
       encoding='utf-8')
tokenized_text = word_tokenize(re.sub('[\s]'," ", document))
stanentities = st.tag(tokenized_text)

Below, I established two lists to hold the values that I extract from the text.  This itemgetter function will check for unique values.  First, I iterate over the extracted entities and see if the objects is a nltk.tree.Tree with a "Person" label.  If it is, and the length is equal to 1 (first or last name only), I append that value to the list. If it's larger, I iterate of the entity tree and pull out the first value only using itemgetter.  Then, I join the values from the list and append it to the destination list.  

# Getting a list out of NLTK's standard NE chunker

In [11]:
from operator import itemgetter

persons = []
organizations = []
locations =[]

for l in entities:
    if isinstance(l,nltk.tree.Tree):
        if l.label() == 'PERSON':
            if len(l)== 1:
                if l[0][0] in persons:
                    pass
                else:
                    persons.append(l[0][0])
            else:
                if " ".join(map(itemgetter(0), l)) in persons:
                    pass
                else:
                    persons.append(" ".join(map(itemgetter(0), l)))
                    
for o in entities:
    if isinstance(o,nltk.tree.Tree):
        if o.label() == 'ORGANIZATION':
            if len(o)== 1:
                if o[0][0] in organizations:
                    pass
                else:
                    organizations.append(o[0][0])
            else:
                if " ".join(map(itemgetter(0), o)) in organizations:
                    pass
                else:
                    organizations.append(" ".join(map(itemgetter(0), o)))
                    
for o in entities:
    if isinstance(o,nltk.tree.Tree):
        if o.label() == 'LOCATION':
            if len(o)== 1:
                if o[0][0] in locations:
                    pass
                else:
                    locations.append(o[0][0])
            else:
                if " ".join(map(itemgetter(0), o)) in locations:
                    pass
                else:
                    locations.append(" ".join(map(itemgetter(0), o)))
                    
                
print persons
print
print
print organizations
print
print
print locations

['Rademacher Averages Matteo Riondato Dept', 'Eli Upfal Dept', 'Subject Descriptors', 'Database Applications Data', 'Algorithms', 'Experimentation Keywords Frequent Itemsets', 'Pattern Mining', 'Rademacher Averages', 'Statistical Learning', 'Permissions', 'Outline', 'Riondato', 'Upfal', 'Chen', 'Chuang', 'Pietracaprina', 'Scheer', 'Wrobel', 'Elomaa', 'Assume', 'B', 'Hence', 'Thm', 'S S.', 'C1', 'D', 'C1 IS', 'X X', 'Si', 'Due', 'Grahne', 'Zhu', 'Unless', 'Frequency Estimation', 'Fast', 'Bartlett', 'Model', 'Mach', 'J. Pei', 'Data Mining', 'Knowl', 'Disc', 'John', 'Static', 'Johnson', 'J. Thaler', 'Space', 'Data Mining Knowl', 'Rademacher', 'Version', 'Learn', 'Machine Learning']


['Computer Science Brown University Providence', 'FIs', 'Frequent Itemsets', 'ACM', 'NSW', 'CIs', 'Parthasarathy', 'fD', 'D', 'TOPK', 'fA', 'SAMPLING', 'AI', 'Thm', 'VS', 'Massarts Lemma', 'RS', 'Euclidean', 'vVS', 'Hoedings', 'esRS', 'Closed Itemsets', 'vS', 'Rademacher Average', 'Closed', 'lnX', 'aX', 'aIS'

In [12]:
from operator import itemgetter

for o in entities:
    if isinstance(o,nltk.tree.Tree):
        if o.label() == 'ORGANIZATION' or o.label() == 'GPE':
            if len(o)>1:
                print " ".join(map(itemgetter(0), o))

Computer Science Brown University Providence
Computer Science Brown University Providence
Frequent Itemsets
Frequent Itemsets
Massarts Lemma
Closed Itemsets
Closed Itemsets
Rademacher Average
SIGKDD Explor
IEEE Trans
ACM Trans
Nature of Statistical
New York
University Press


I tried to iterate over the extracted list of entities to get a better break between person's and their university name.  

In [13]:
tokens = [nltk.word_tokenize(l) for l in persons]
fin = [nltk.chunk.ne_chunk(nltk.pos_tag(l)) for l in tokens]
fin;

In [14]:
new =[word_tokenize(l) for l in persons]
stan = [st.tag(l) for l in new]
stan;

# Creating lists of named entities from Stanford's NER model

This function looks though an extracted stanford ner list, and finds continuous entitiy labels.  This should create first name, last name records of entities.  

In [15]:
def get_continuous_chunks(tagged_sent):
    continuous_chunk = []
    current_chunk = []

    for token, tag in tagged_sent:
        if tag != "O":
            current_chunk.append((token, tag))
        else:
            if current_chunk: # if the current chunk is not empty
                continuous_chunk.append(current_chunk)
                current_chunk = []
    # Flush the final current_chunk into the continuous_chunk, if any.
    if current_chunk:
        continuous_chunk.append(current_chunk)
    return continuous_chunk

ne_tagged_sent = [('Rami', 'PERSON'), ('Eid', 'PERSON'), ('is', 'O'), ('studying', 'O'), ('at', 'O'), ('Stony', 'ORGANIZATION'), ('Brook', 'ORGANIZATION'), ('University', 'ORGANIZATION'), ('in', 'O'), ('NY', 'LOCATION')]

named_entities = get_continuous_chunks(stanentities)
named_entities_str = [" ".join([token for token, tag in ne]) for ne in named_entities]
named_entities_str_tag = [(" ".join([token for token, tag in ne]), ne[0][1]) for ne in named_entities]


In [16]:
named_entities = get_continuous_chunks(stanentities)
named_entities_str = [" ".join([token for token, tag in ne]) for ne in named_entities]
named_entities_str_tag

[(u'Rademacher Averages Matteo Riondato Dept', u'ORGANIZATION'),
 (u'Computer Science Brown University Providence', u'ORGANIZATION'),
 (u'Eli Upfal Dept', u'ORGANIZATION'),
 (u'Computer Science Brown University Providence', u'ORGANIZATION'),
 (u'FIs', u'ORGANIZATION'),
 (u'Rademacher', u'PERSON'),
 (u'Database Applications Data mining General Terms Algorithms',
  u'ORGANIZATION'),
 (u'Rademacher', u'PERSON'),
 (u'August 10-13', u'DATE'),
 (u'2015', u'DATE'),
 (u'Sydney', u'LOCATION'),
 (u'NSW', u'ORGANIZATION'),
 (u'Australia', u'LOCATION'),
 (u'2015', u'DATE'),
 (u'$ 15.00', u'MONEY'),
 (u'FIs', u'ORGANIZATION'),
 (u'FIs', u'ORGANIZATION'),
 (u'Rademacher', u'PERSON'),
 (u'Rademacher', u'PERSON'),
 (u'Rademacher', u'PERSON'),
 (u'FIs', u'ORGANIZATION'),
 (u'FIs', u'ORGANIZATION'),
 (u'Parthasarathy', u'PERSON'),
 (u'Scheer', u'PERSON'),
 (u'Wrobel', u'PERSON'),
 (u'Scheer', u'PERSON'),
 (u'Wrobel', u'PERSON'),
 (u'FIs', u'ORGANIZATION'),
 (u'Pietracaprina', u'ORGANIZATION'),
 (u'Chern

In [17]:
compare=[]
for l,m in named_entities_str_tag:
    l=re.sub('(\])'," ",l)
    if m == 'PERSON':
        if l in compare:
            pass
        else:
            compare.append(l)
    else:
        pass
compare

[u'Rademacher',
 u'Parthasarathy',
 u'Scheer',
 u'Wrobel',
 u'Grahne',
 u'  R. Agrawal',
 u'R. Srikant',
 u'T. Imieliski',
 u'A. Swami',
 u'  B. Gu',
 u'B. Liu',
 u'F. Hu',
 u'H. Liu',
 u'  P. L. Bartlett',
 u'S. Boucheron',
 u'G. Lugosi',
 u'  S. Boucheron',
 u'O. Bousquet',
 u'  V. T. Chakaravarthy',
 u'V. Pandit',
 u'Y. Sabharwal',
 u'  B. Chen',
 u'P. Haas',
 u'P. Scheuermann',
 u'K.-T. Chuang',
 u'M.-S. Chen',
 u'M. Kriinen',
 u'B. Goethals',
 u'M. J. Zaki',
 u'  G. Grahne',
 u'J. Zhu',
 u'  J. Han',
 u'J. Pei',
 u'Y. Yin',
 u'H. Cheng',
 u'D. Xin',
 u'X. Yan',
 u'G. H. John',
 u'P. Langley',
 u'S. G. Johnson',
 u'V. Koltchinskii',
 u'M. Mitzenmacher',
 u'J. Thaler',
 u'J. Ullman',
 u'S. Parthasarathy',
 u'N. Pasquier',
 u'Y. Bastide',
 u'R. Taouil',
 u'L. Lakhal',
 u'A. Pietracaprina',
 u'M. Riondato',
 u'E. Upfal',
 u'D. Jensen',
 u'T. Oates',
 u'  M. Riondato',
 u'R. Fonseca',
 u'F. Vandin',
 u'V. N. Vapnik',
 u'G. I. Webb',
 u'T. Scheer',
 u'S. Wrobel',
 u'S. Ben-David',
 u'H.

In [18]:
list1 = range(10)

In [19]:
list2 = [i for i in xrange(7,17,1)]


In [20]:
set(list1) & set(list2)

{7, 8, 9}

In [21]:
def parts_of_speech(corpus):
    "returns named entity chunks in a given text"
    tagged = nltk.pos_tag(nltk.word_tokenize(corpus))
    entities = nltk.chunk.ne_chunk(tagged)
    # Another entity extractor
    st = StanfordNERTagger('/Users/linwood/stanford-corenlp-full-2015-04-20/classifiers/english.muc.7class.distsim.crf.ser.gz',
           '/Users/linwood/stanford-corenlp-full-2015-04-20/stanford-corenlp-3.5.2.jar',
           encoding='utf-8')
    tokenized_text = word_tokenize(corpus)
    stanentities = st.tag(tokenized_text)
    return entities
def find_entities(chunks):
    "given list of tagged parts of speech, returns unique named entities"

    def traverse(tree):
        "recursively traverses an nltk.tree.Tree to find named entities"
        entity_names = []
    
        if hasattr(tree, 'node') and tree.node:
            if tree.node == 'NE':
                entity_names.append(' '.join([child[0] for child in tree]))
            else:
                for child in tree:
                    entity_names.extend(traverse(child))
    
        return entity_names
    
    named_entities = []
    
    for chunk in chunks:
        entities = sorted(list(set([word for tree in chunk
                            for word in traverse(tree)])))
        for e in entities:
            if e not in named_entities:
                named_entities.append(e)
    return named_entities

# Extracting entities and creating lists using Polyglot

In [22]:
from polyglot.text import Text
e=Text(re.sub('[\s]'," ",document[:10000])).entities

This code iterates over the polyglot extracted entities and creates a list of person, locations, and organizations

In [23]:
import itertools
import unicodedata

def extraction(corpus):
    
    # extract entities from a single string; remove whitespace characters
    try:
        e = Text(re.sub('[\s]'," ",corpus)).entities
    except:
        pass #e = Text(re.sub("(r'(x0)'," ","(re.sub('[\s]'," ",corpus)))).entities
    
    current_person =[]
    persons =[]
    current_org=[]
    organizations=[]
    current_loc=[]
    locations=[]

    for l in e:
        if l.tag == 'I-PER':
            for m in l:
                current_person.append(unicodedata.normalize('NFKD', m).encode('ascii','ignore'))
            else:
                    if current_person: # if the current chunk is not empty
                        persons.append(" ".join(current_person))
                        current_person = []
        elif l.tag == 'I-ORG':
            for m in l:
                current_org.append(unicodedata.normalize('NFKD', m).encode('ascii','ignore'))
            else:
                    if current_org: # if the current chunk is not empty
                        organizations.append(" ".join(current_org))
                        current_org = []
        elif l.tag == 'I-LOC':
            for m in l:
                current_loc.append(unicodedata.normalize('NFKD', m).encode('ascii','ignore'))
            else:
                    if current_loc: # if the current chunk is not empty
                        locations.append(" ".join(current_loc))
                        current_loc = []
    results = {}
    results['persons']=persons
    results['organizations']=organizations
    results['locations']=locations
    
    return results

In [24]:
extraction(document)['persons']

['Matteo Riondato',
 'matteo',
 'cs.brown.edu Eli Upfal',
 'KDD15',
 'Chen',
 'Chuang',
 'thm',
 'Pietracaprina',
 'Scheer',
 'Scheer',
 'Pietracaprina et',
 'Cherno',
 'David',
 'Elomaa',
 'aX',
 'X',
 'Si1',
 'Liberty',
 'Riondato',
 'Riondato',
 'Grahne',
 'Zhu',
 'matteo',
 'radeprogrfi.tar.bz2',
 'errorrelative',
 'error1012',
 'Agrawal',
 'Agrawal',
 'Mining',
 'Liu',
 'Hu',
 'Liu',
 'Eciently',
 'Bartlett',
 'Lugosi',
 'Lugosi',
 'Pandit',
 'Chen',
 'Haas',
 'Chuang',
 'Chen',
 'Yang',
 'Kriinen',
 'J',
 'Zaki',
 'SIGKDD',
 'Grahne',
 'Zhu',
 'Han',
 'Pei',
 'Han',
 'Cheng',
 'John',
 'Johnson',
 'J',
 'Thaler',
 'Ullman',
 'CoRR',
 'Jensen',
 'Oates',
 'Ecient',
 'Fonseca',
 'Rademacher',
 'Webb',
 'Scheer',
 'Ben',
 'David']

In [25]:
document;

In [26]:
regexp = re.compile("REFERENCES(.*)$")
references = Text(regexp.search(re.sub('[\s]'," ",document)).group(1)).entities

In [27]:
regexp1 = re.compile("REFERENCES(.*)$")
references = Text(regexp.search(re.sub('[\s]'," ",document)).group(1)).entities

In [28]:
len(extraction(regexp.search(re.sub('[\s]'," ",document)).group(1))['persons'])

41

# Truth Sets to test extraction accuracy

In [29]:
#p19.pdf

p19pdf_authors=['Tim Althoff','Xin Luna Dong','Kevin Murphy','Safa Alai','Van Dang','Wei Zhang']
p19pdf_author_organizations=['Computer Science Department','Stanford University','Google']
p19pdf_author_locations=['Stanford, CA','Stanford','CA','Google','1600 Amphitheatre Parkway, Mountain View, CA 94043','1600 Amphitheatre Parkway','Mountain View']

p19pdf_references_authors =['A. Ahmed', 'C. H. Teo', 'S. Vishwanathan','A. Smola','J. Allan', 'R. Gupta', 'V. Khandelwal',
                           'D. Graus', 'M.-H. Peetz', 'D. Odijk', 'O. de Rooij', 'M. de Rijke','T. Huet', 'J. Biega', 
                            'F. M. Suchanek','H. Ji', 'T. Cassidy', 'Q. Li','S. Tamang', 'A. Kannan', 'S. Baker', 'K. Ramnath', 
                            'J. Fiss', 'D. Lin', 'L. Vanderwende',  'R. Ansary', 'A. Kapoor', 'Q. Ke', 'M. Uyttendaele',
                           'S. M. Katz','A. Krause','D. Golovin','J. Leskovec', 'A. Krause', 'C. Guestrin', 'C. Faloutsos', 
                            'J. VanBriesen','N. Glance','J. Li','C. Cardie','J. Li','C. Cardie','C.-Y. Lin','H. Lin','J. A. Bilmes'
                           'X. Ling','D. S. Weld', 'A. Mazeika', 'T. Tylenda','G. Weikum','M. Minoux', 'G. L. Nemhauser', 'L. A. Wolsey',
                            'M. L. Fisher','R. Qian','D. Shahaf', 'C. Guestrin','E. Horvitz','T. Althoff', 'X. L. Dong', 'K. Murphy', 'S. Alai',
                            'V. Dang','W. Zhang','R. A. Baeza-Yates', 'B. Ribeiro-Neto', 'D. Shahaf', 'J. Yang', 'C. Suen', 'J. Jacobs', 'H. Wang', 'J. Leskovec',
                           'W. Shen', 'J. Wang', 'J. Han','D. Bamman', 'N. Smith','K. Bollacker', 'C. Evans', 'P. Paritosh', 'T. Sturge', 'J. Taylor',
                           'R. Sipos', 'A. Swaminathan', 'P. Shivaswamy', 'T. Joachims','K. Sprck Jones','G. Calinescu', 'C. Chekuri', 'M. Pl','J. Vondrk',
                           'F. M. Suchanek', 'G. Kasneci','G. Weikum', 'J. Carbonell' ,'J. Goldstein','B. Carterette', 'P. N. Bennett', 'D. M. Chickering',
                            'S. T. Dumais','A. Dasgupta', 'R. Kumar','S. Ravi','Q. X. Do', 'W. Lu', 'D. Roth','X. Dong', 'E. Gabrilovich', 'G. Heitz', 'W. Horn', 
                            'N. Lao', 'K. Murphy',  'T. Strohmann', 'S. Sun','W. Zhang', 'M. Dubinko', 'R. Kumar', 'J. Magnani', 'J. Novak', 'P. Raghavan','A. Tomkins',
                           'U. Feige','F. M. Suchanek','N. Preda','R. Swan','J. Allan', 'T. Tran', 'A. Ceroni', 'M. Georgescu', 'K. D. Naini', 'M. Fisichella',
                           'T. A. Tuan', 'S. Elbassuoni', 'N. Preda','G. Weikum','Y. Wang', 'M. Zhu', 'L. Qu', 'M. Spaniol', 'G. Weikum',
                           'G. Weikum', 'N. Ntarmos', 'M. Spaniol', 'P. Triantallou', 'A. A. Benczr',  'S. Kirkpatrick', 'P. Rigaux','M. Williamson',
                           'X. W. Zhao', 'Y. Guo', 'R. Yan', 'Y. He','X. Li']
p19pdf_allauthors=['Tim Althoff','Xin Luna Dong','Kevin Murphy','Safa Alai','Van Dang','Wei Zhang','A. Ahmed', 'C. H. Teo', 'S. Vishwanathan','A. Smola','J. Allan', 'R. Gupta', 'V. Khandelwal',
                           'D. Graus', 'M.-H. Peetz', 'D. Odijk', 'O. de Rooij', 'M. de Rijke','T. Huet', 'J. Biega', 
                            'F. M. Suchanek','H. Ji', 'T. Cassidy', 'Q. Li','S. Tamang', 'A. Kannan', 'S. Baker', 'K. Ramnath', 
                            'J. Fiss', 'D. Lin', 'L. Vanderwende',  'R. Ansary', 'A. Kapoor', 'Q. Ke', 'M. Uyttendaele',
                           'S. M. Katz','A. Krause','D. Golovin','J. Leskovec', 'A. Krause', 'C. Guestrin', 'C. Faloutsos', 
                            'J. VanBriesen','N. Glance','J. Li','C. Cardie','J. Li','C. Cardie','C.-Y. Lin','H. Lin','J. A. Bilmes'
                           'X. Ling','D. S. Weld', 'A. Mazeika', 'T. Tylenda','G. Weikum','M. Minoux', 'G. L. Nemhauser', 'L. A. Wolsey',
                            'M. L. Fisher','R. Qian','D. Shahaf', 'C. Guestrin','E. Horvitz','T. Althoff', 'X. L. Dong', 'K. Murphy', 'S. Alai',
                            'V. Dang','W. Zhang','R. A. Baeza-Yates', 'B. Ribeiro-Neto', 'D. Shahaf', 'J. Yang', 'C. Suen', 'J. Jacobs', 'H. Wang', 'J. Leskovec',
                           'W. Shen', 'J. Wang', 'J. Han','D. Bamman', 'N. Smith','K. Bollacker', 'C. Evans', 'P. Paritosh', 'T. Sturge', 'J. Taylor',
                           'R. Sipos', 'A. Swaminathan', 'P. Shivaswamy', 'T. Joachims','K. Sprck Jones','G. Calinescu', 'C. Chekuri', 'M. Pl','J. Vondrk',
                           'F. M. Suchanek', 'G. Kasneci','G. Weikum', 'J. Carbonell' ,'J. Goldstein','B. Carterette', 'P. N. Bennett', 'D. M. Chickering',
                            'S. T. Dumais','A. Dasgupta', 'R. Kumar','S. Ravi','Q. X. Do', 'W. Lu', 'D. Roth','X. Dong', 'E. Gabrilovich', 'G. Heitz', 'W. Horn', 
                            'N. Lao', 'K. Murphy',  'T. Strohmann', 'S. Sun','W. Zhang', 'M. Dubinko', 'R. Kumar', 'J. Magnani', 'J. Novak', 'P. Raghavan','A. Tomkins',
                           'U. Feige','F. M. Suchanek','N. Preda','R. Swan','J. Allan', 'T. Tran', 'A. Ceroni', 'M. Georgescu', 'K. D. Naini', 'M. Fisichella',
                           'T. A. Tuan', 'S. Elbassuoni', 'N. Preda','G. Weikum','Y. Wang', 'M. Zhu', 'L. Qu', 'M. Spaniol', 'G. Weikum',
                           'G. Weikum', 'N. Ntarmos', 'M. Spaniol', 'P. Triantallou', 'A. A. Benczr',  'S. Kirkpatrick', 'P. Rigaux','M. Williamson',
                           'X. W. Zhao', 'Y. Guo', 'R. Yan', 'Y. He','X. Li']

print "There are %r authors" % len(p19pdf_authors)
print  # white space
print "There are %r author organizations" %len(p19pdf_author_organizations)
print 
print "There are %r author locations" % len(p19pdf_author_locations)
print  
print "There are %r authors in the references" %len(p19pdf_references_authors)

There are 6 authors

There are 3 author organizations

There are 7 author locations

There are 152 authors in the references


In [30]:
#p29.pdf

p29pdf_authors=['Laurent Amsaleg','Stéphane Girard','Oussama Chelly','Teddy Furon','Michael E. Houle','Ken-ichi Kawarabayashi',
               'Michael Nett']
p29pdf_author_organizations=['Equipe LINKMEDIA','Campus Universitaire de Beaulieu','CNRS/IRISA Rennes','National Institute of Informatics',
                             'Equipe MISTIS INRIA','Google']
p29pdf_author_locations=['Campus Universitaire de Beaulieu','35042 Rennes Cedex, France','France','-1-2 Hitotsubashi, Chiyoda-ku Tokyo 101-8430, Japan',
                        'Japan','6-10-1 Roppongi, Minato-ku Tokyo 106-6126','Inovallée, 655, Montbonnot 38334 Saint-Ismier Cedex','Tokyo']

p29pdf_references_authors =['A. A. Balkema','L. de Haan','N. Bingham', 'C. Goldie','J. Teugels','N. Boujemaa', 'J. Fauqueur', 'M. Ferecatu', 'F. Fleuret',
                            'V. Gouet', 'B. LeSaux','H. Sahbi','C. Bouveyron', 'G. Celeux', 'S. Girard','J. Bruske', 'G. Sommer',
                           'F. Camastra','A. Vinciarelli','S. Coles','J. Costa' ,'A. Hero','T. de Vries', 'S. Chawla','M. E. Houle',
                           'R. A. Fisher','L. H. C. Tippett','M. I. Fraga Alves', 'L. de Haan','T. Lin','M. I. Fraga Alves', 'M. I. Gomes','L. de Haan',
                           'B. V. Gnedenko',' A. Gupta', 'R. Krauthgamer','J. R. Lee','A. Gupta', 'R. Krauthgamer','J. R. Lee','M. Hein','J.-Y. Audibert',
                           'B. M. Hill','M. E. Houle','M. E. Houle','M. E. Houle','M. E. Houle', 'H. Kashima', 'M. Nett','M. E. Houle', 'X. Ma', 'M. Nett',
                            'V. Oria','M. E. Houle', 'X. Ma', 'V. Oria','J. Sun','M. E. Houle','M. Nett','H. Jegou', 'R. Tavenard', 'M. Douze','L. Amsaleg',
                           'I. Jollie','D. R. Karger','M. Ruhl','J. Karhunen','J. Joutsensalo','Y. LeCun', 'L. Bottou', 'Y. Bengio', 'P. Haner',
                           'J. Pickands, III','C. R. Rao','S. T. Roweis','L. K. Saul','A. Rozza', 'G. Lombardi', 'C. Ceruti', 'E. Casiraghi', 'P. Campadelli',
                           'B. Scholkopf', 'A. J. Smola','K.-R. Muller','U. Shaft','R. Ramakrishnan',' F. Takens','J. Tenenbaum', 'V. D. Silva','J. Langford',
                           'J. B. Tenenbaum', 'V. De Silva','J. C. Langford','J. B. Tenenbaum', 'V. De Silva','J. C. Langford','J. Venna','S. Kaski',
                           'P. Verveer','R. Duin','J. von Brunken', 'M. E. Houle', 'A. Zimek','J. von Brunken', 'M. E. Houle','A. Zimek']

p29pdf_allauthors=['Laurent Amsaleg','Stéphane Girard','Oussama Chelly','Teddy Furon','Michael E. Houle','Ken-ichi Kawarabayashi',
               'Michael Nett','A. A. Balkema','L. de Haan','N. Bingham', 'C. Goldie','J. Teugels','N. Boujemaa', 'J. Fauqueur', 'M. Ferecatu', 'F. Fleuret',
                            'V. Gouet', 'B. LeSaux','H. Sahbi','C. Bouveyron', 'G. Celeux', 'S. Girard','J. Bruske', 'G. Sommer',
                           'F. Camastra','A. Vinciarelli','S. Coles','J. Costa' ,'A. Hero','T. de Vries', 'S. Chawla','M. E. Houle',
                           'R. A. Fisher','L. H. C. Tippett','M. I. Fraga Alves', 'L. de Haan','T. Lin','M. I. Fraga Alves', 'M. I. Gomes','L. de Haan',
                           'B. V. Gnedenko',' A. Gupta', 'R. Krauthgamer','J. R. Lee','A. Gupta', 'R. Krauthgamer','J. R. Lee','M. Hein','J.-Y. Audibert',
                           'B. M. Hill','M. E. Houle','M. E. Houle','M. E. Houle','M. E. Houle', 'H. Kashima', 'M. Nett','M. E. Houle', 'X. Ma', 'M. Nett',
                            'V. Oria','M. E. Houle', 'X. Ma', 'V. Oria','J. Sun','M. E. Houle','M. Nett','H. Jegou', 'R. Tavenard', 'M. Douze','L. Amsaleg',
                           'I. Jollie','D. R. Karger','M. Ruhl','J. Karhunen','J. Joutsensalo','Y. LeCun', 'L. Bottou', 'Y. Bengio', 'P. Haner',
                           'J. Pickands, III','C. R. Rao','S. T. Roweis','L. K. Saul','A. Rozza', 'G. Lombardi', 'C. Ceruti', 'E. Casiraghi', 'P. Campadelli',
                           'B. Scholkopf', 'A. J. Smola','K.-R. Muller','U. Shaft','R. Ramakrishnan',' F. Takens','J. Tenenbaum', 'V. D. Silva','J. Langford',
                           'J. B. Tenenbaum', 'V. De Silva','J. C. Langford','J. B. Tenenbaum', 'V. De Silva','J. C. Langford','J. Venna','S. Kaski',
                           'P. Verveer','R. Duin','J. von Brunken', 'M. E. Houle', 'A. Zimek','J. von Brunken', 'M. E. Houle','A. Zimek']


print "There are %r authors" % len(p29pdf_authors)
print  # white space
print "There are %r author organizations" %len(p29pdf_author_organizations)
print 
print "There are %r author locations" % len(p29pdf_author_locations)
print  
print "There are %r authors in the references" %len(p29pdf_references_authors)

There are 7 authors

There are 6 author organizations

There are 8 author locations

There are 106 authors in the references


In [31]:
len(set(compare) & set(p29pdf_allauthors))

0

In [32]:
len(p29pdf_allauthors)

113

In [33]:
re.sub('[\s]'," ",document)[9300:10500]
#regexp.search(re.sub('[\s]'," ",document)).group(1)[4900:6000]

' [9] on learning two-level decision trees, whose settings and problem are very dierent from the ones we study.  3. DEFINITIONS AND PRELIMINARIES Let I be a set of items with an arbitrary xed total order <. A transaction is a subset of I, and a transactional dataset is a collection of transactions. An itemset is a set of items that appear together in a transaction (i.e., a subset of a transaction). Given an itemset A and a transaction  s.t. A  , we say that A appears or is contained in  and that  contains A. The support set TD(A) of A in D is the subset of transactions in D that contain the itemset A, and the frequency of itemset A in dataset D is the fraction of transactions of D that contain A:  fD(A) = |TD(A)|/|D| .  Given a frequency threshold   (0, 1], the set FI(D,I, ) of Frequent Itemsets (FIs) in D w.r.t.  is the collection of all itemsets with frequency at least  in D:  FI(D,I, ) = {(A, fD(A)) : A  I  fD(A)  } .  Similarly, let f least k itemsets have frequency at least f set 

In [34]:
extraction(document)

{'locations': ['RI',
  'RI',
  'Sydney',
  'NSW',
  'Australia',
  'Wrobel',
  'Wrobel',
  'CIs',
  'Ca',
  'Thm',
  'Thm',
  'Thm',
  'Thm',
  'Liberty et',
  'Thm',
  'Thm',
  'PARMA',
  'PARMA',
  'sion',
  'MapReduce',
  'New York',
  'NY',
  'USA'],
 'organizations': ['Computer Science Brown University Providence',
  'Computer Science Brown University Providence',
  'FIs',
  'FIs',
  'ACM',
  'FIs',
  'CIs',
  'CIs',
  'FIs',
  'FIs',
  'Def',
  'Def',
  'Def',
  '2n2',
  'CIs',
  'SA',
  'B SA',
  'CIs',
  'CIs',
  'CIs',
  'aIS',
  'aIS',
  'aIS',
  'ACa',
  'aIS',
  'ACa',
  'nal',
  'Si1',
  'AMD',
  'GNU',
  'Linux',
  'FIs',
  'Def',
  'FI',
  'BMS',
  'BMS',
  'NSF',
  'Swami',
  'Yan',
  'Data Mining',
  'Langley',
  'IEEE',
  'Liberty',
  'Mining',
  'Mining',
  'PARMA',
  'ACM',
  'Springer',
  'Verlag',
  'University Press'],
 'persons': ['Matteo Riondato',
  'matteo',
  'cs.brown.edu Eli Upfal',
  'KDD15',
  'Chen',
  'Chuang',
  'thm',
  'Pietracaprina',
  'Scheer',
 

In [35]:
class entities(object):
  def __init__(self):
    self.persons = extraction(document)['persons']
    self.organizations = extraction(document)['organizations']

my_shape = entities()
my_shape.persons

['Matteo Riondato',
 'matteo',
 'cs.brown.edu Eli Upfal',
 'KDD15',
 'Chen',
 'Chuang',
 'thm',
 'Pietracaprina',
 'Scheer',
 'Scheer',
 'Pietracaprina et',
 'Cherno',
 'David',
 'Elomaa',
 'aX',
 'X',
 'Si1',
 'Liberty',
 'Riondato',
 'Riondato',
 'Grahne',
 'Zhu',
 'matteo',
 'radeprogrfi.tar.bz2',
 'errorrelative',
 'error1012',
 'Agrawal',
 'Agrawal',
 'Mining',
 'Liu',
 'Hu',
 'Liu',
 'Eciently',
 'Bartlett',
 'Lugosi',
 'Lugosi',
 'Pandit',
 'Chen',
 'Haas',
 'Chuang',
 'Chen',
 'Yang',
 'Kriinen',
 'J',
 'Zaki',
 'SIGKDD',
 'Grahne',
 'Zhu',
 'Han',
 'Pei',
 'Han',
 'Cheng',
 'John',
 'Johnson',
 'J',
 'Thaler',
 'Ullman',
 'CoRR',
 'Jensen',
 'Oates',
 'Ecient',
 'Fonseca',
 'Rademacher',
 'Webb',
 'Scheer',
 'Ben',
 'David']

# Code to extract emails

In [43]:
tuple(get_emails(document))

('matteo@cs.brown.edu', 'eli@cs.brown.edu', 'ermissions@acm.org')

# Code to get only the Title and Author Names

In [44]:
p=re.compile('(.*)(?=ABSTRACT)')
p.search(re.sub('[\s]'," ",document)).group(1)

'Mining Frequent Itemsets through Progressive Sampling  with Rademacher Averages  Matteo Riondato  Dept. of Computer Science  Brown University  Providence, RI 02912  matteo@cs.brown.edu  Eli Upfal  Dept. of Computer Science  Brown University  Providence, RI 02912 eli@cs.brown.edu  '

# Code to get Title Only (or most of it)

In [45]:
p=re.compile('(.+)(\\n\\n)')
q=re.compile('(?<=\\n\\n)(.+?)(?=\\n\\n)')
#p.search(document).group(1)+" "+q.search(document).group(1)
p.search(document).group(1)

'Mining Frequent Itemsets through Progressive Sampling'

# Code to get Abstract only

In [46]:
p= re.compile('(?<=ABSTRACT)(.+)(?=Categories and Subject Descriptors)')
p.search(re.sub('[\s]'," ",document)).group(1)

' We present an algorithm to extract an high-quality approx- imation of the (top-k) Frequent itemsets (FIs) from ran- dom samples of a transactional dataset. With high prob- ability the approximation is a superset of the FIs, and no itemset with frequency much lower than the threshold is in- cluded in it. The algorithm employs progressive sampling, with a stopping condition based on bounds to the empirical Rademacher average, a key concept from statistical learning theory. The computation of the bounds uses characteris- tic quantities that can be obtained eciently with a sin- gle scan of the sample. Therefore, evaluating the stopping condition is fast, and does not require an expensive mining of each sample. Our experimental evaluation conrms the practicality of our approach on real datasets, outperforming approaches based on one-shot static sampling.  '

# Code to get keywords only

In [47]:
p= re.compile('(?<=Keywords)(.+)(?=INTRODUCTION)')
p.search(re.sub('[\s]'," ",document)).group(1)

' Frequent Itemsets; Pattern Mining; Rademacher Averages; Sampling; Statistical Learning Theory  1.  '

# Code to get Categories and Subject Descriptors

In [48]:
p= re.compile('(?<=Categories and Subject Descriptors)(.+)(?=Keywords)')
re.findall('\[(.*?)\]',p.search(re.sub('[\s]'," ",document)).group(1))

['Database Management']

# Code to get Body only

In [49]:
p= re.compile('(?<=INTRODUCTION)(.+)(?=ACKNOWLEDGMENTS)')
p.search(re.sub('[\s]'," ",document)).group(1)

'  The task of Frequent Itemsets (FIs) mining is to extract all sets of items that appear in at least a fraction  of a transac- tional dataset D, or the k most frequent set of items [2]. It is a fundamental primitive of knowledge discovery and is use- ful, among the others, for market basket analysis, inference, classication, and network management [13]. Exact algo- rithms to mine FIs have since long been available but their  Permission to make digital or hard copies of all or part of this work for personal or classroom use is granted without fee provided that copies are not made or distributed for prot or commercial advantage and that copies bear this notice and the full cita- tion on the rst page. Copyrights for components of this work owned by others than ACM must be honored. Abstracting with credit is permitted. To copy otherwise, or re- publish, to post on servers or to redistribute to lists, requires prior specic permission and/or a fee. Request permissions from Permissions@acm.o

# Code to get CONCLUSION only

In [50]:
p= re.compile('(?<=CONCLUSION)(.+)(?=ACKNOWLEDGMENTS)')
p.search(re.sub('[\s]'," ",document)).group(1)

'S  We present an algorithm for extracting a high-quality ap- proximation of the collection of FIs with probabilistic guar- antees. The algorithm employs progressive sampling with a stopping condition that relies on bounding the conditional Rademacher average of the problem using easy-to-compute characteristic quantities of the sample. The stopping con- dition can therefore be evaluated very eciently without the need to perform an expensive in-depth mining of the frequent itemsets in the sample at each step. To our knowl- edge this is the rst work that uses Rademacher averages in a knowledge discovery setting. The experimental results conrm that the algorithm is extremely successful at stop- ping fast at early iterations, and allows to extract very high- quality approximation of the collection of FIs. Among the possible directions for future work, it would be particularly interesting to better study the trade-o between the compu- tational complexity of the stopping condition and its ab

# Code to get References only

In [51]:
p= re.compile('(?<=REFERENCES)(.+)')
p.search(re.sub('[\s]'," ",document)).group(1)

'  [1] R. Agrawal and R. Srikant. Fast algorithms for mining  association rules in large databases. VLDB 94.  [2] R. Agrawal, T. Imieliski, and A. Swami. Mining asso- ciation rules between sets of items in large databases. SIGMOD Rec., 22:207216, June 1993.  [3] B. Gu, B. Liu, F. Hu, and H. Liu. Eciently determin- ing the starting sample size for progressive sampling. ECML01.  [4] P. L. Bartlett, S. Boucheron, and G. Lugosi. Model selection and error estimation. Mach. Learn., 48:85 113, 2002.  [5] S. Boucheron, O. Bousquet, and G. Lugosi. Theory of classication : A survey of some recent advances. ESAIM: Probability and Statistics, 9:323375, 2005.  [6] V. T. Chakaravarthy, V. Pandit, and Y. Sabharwal. Analysis of sampling techniques for association rule mining. ICDT09.  [7] B. Chen, P. Haas, and P. Scheuermann. A new two- phase sampling based algorithm for discovering associ- ation rules. KDD02.  [8] K.-T. Chuang, M.-S. Chen, and W.-C. Yang. Progres- sive sampling for association rules 

# Code to count the number of references

In [52]:
'''
create a match of all references by counting the number of integers enclosed by brackets (i.e. '[2]').  

This is how references are labeled in the research papers
'''

len(re.findall('\[(.*?)\]',regexp.search(re.sub('[\s]'," ",document)).group(1)))

30